In [ ]:
%reload_ext autoreload
%autoreload 2

# IDQL 

> IDQL class
> 
>Title: Implicit Q-Learning with Diffusion Policy
>Author: Binjian Xin
>Date created: 2024/06/20
>Last modified: 2024/06/20
>Description: Agent from IDQL repo
>
>
>Title: IDQL: Implicit Q-learning as an Actor-Critic Meithod with Diffusion Policies
>Author: Philippe Hansen-Estruch, Ilya Kostrikov, Michael Janner, Jakub Grudzien Kuba, Sergey levine 
>Description: Implementing IDQL algorithm on VEOS.


In [ ]:
#| default_exp agent.idql

In [ ]:
#| export
import logging
import os
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Tuple
import numpy as np
import pandas as pd
import tensorflow as tf
from typeguard import check_type
import jax
import jax.numpy as jnp

In [ ]:
#| export
from tspace.agent.utils.hyperparams import HyperParamDDPG, HyperParamRDPG, HyperParamIDQL
from tspace.storage.buffer.dask import DaskBuffer
from tspace.storage.buffer.mongo import MongoBuffer  # type: ignore
from tspace.data.core import PoolQuery  # type: ignore
from tspace.data.time import veos_lifetime_end_date, veos_lifetime_start_date
from tspace.data.core import PoolQuery  # type: ignore
from tspace.data.time import veos_lifetime_end_date, veos_lifetime_start_date

In [ ]:
#| export
from tspace.agent.dpg import DPG  # type: ignore

In [ ]:
#| export
from tspace.config.vehicles import Truck, TruckInCloud, trucks_by_id
from tspace.config.drivers import Driver
import logging
from typing import Union
from tspace.data.core import (
    RE_RECIPEKEY,
    ActionSpecs,
    ObservationMetaCloud,
    ObservationMetaECU,
    RewardSpecs,
    StateSpecsCloud,
    StateSpecsECU,
    get_filemeta_config,
)


In [ ]:
#| export
from jaxrl5.agents import DDPMIQLLearner
from jaxrl5.types import DataType
from flax.core import FrozenDict
import gymnasium as gym

/Users/x/.pyenv/versions/miniconda3-3.11-24.1.2-0/envs/tspace/lib/python3.11/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/Users/x/.pyenv/versions/miniconda3-3.11-24.1.2-0/envs/tspace/lib/python3.11/site-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):


In [ ]:
#| export
DatasetDict = dict[str, DataType]

In [ ]:
#| export
@dataclass
class IDQL(DPG):
    """IDQL agent for VEOS.

    Abstracts:

        data interface:
            - pool in mongodb
            - buffer in memory (numpy array)
        model interface:
            - actor networkVj
            - critic network


    Attributes:

        idql_net: actor network
        _ckpt_idql_dir: checkpoint directory for critic

    """

    # Following are derived
    idql_net: Optional[DDPMIQLLearner] = None  # actor_net_default
    action_space: Optional[gym.spaces.Space] = None  # action_space_default
    observation_space: Optional[gym.spaces.Space] = None  # action_space_default
    _ckpt_idql_dir: Optional[Path] = None  # Path("")

    def __post_init__(
        self,
    ):
        """initialize the rdpg agent.

        args:
            truck.ObservationNumber (int): dimension of the state space.
            padding_value (float): value to pad the state with, impossible value for observation, action or re
        """

        self.logger = self.logger.getChild("eos").getChild(self.__str__())
        self.logger.propagate = True

        super().__post_init__()  # call DPG post_init for pool init and plot init
        self.coll_type = "EPISODE"
        self.hyper_param = HyperParamIDQL(
            HiddenDimension=256,
            PaddingValue=-10000,
            tbptt_k1=200,
            tbptt_k2=200,
            BatchSize=4,
            NStates=self.truck.observation_numel,
            NActions=self.truck.torque_flash_numel,
            ActionBias=self.truck.torque_bias,
            NLayerActor=2,
            NLayerCritic=2,
            Gamma=0.99,
            TauActor=0.005,
            TauCritic=0.005,
            ActorLR=0.001,
            CriticLR=0.001,
            CkptInterval=5,
        )

        self.buffer.query = PoolQuery(
            vehicle=self.truck.vid,
            driver=self.driver.pid,
            episodestart_start=veos_lifetime_start_date,
            episodestart_end=veos_lifetime_end_date,
            seq_len_from=1,  # from 10  # sample sequence with a length from 1 to 200
            seq_len_to=self.hyper_param.tbptt_k1 + 100,  # to 300
        )
        self.buffer.pool.query = self.buffer.query

        # actor network (w/ target network)
        self.init_checkpoint()
        
        self.action_space = gym.spaces.Box(
            low=0.0, high=1.0, shape=(self.truck.torque_flash_numel,), dtype=np.float32
        ) # action space is 4*17 matrix
        self.observation_space = gym.spaces.Box(
            low=0.0, high=50.0, shape=(self.truck.observation_numel,), dtype=np.float32
        )  # observation space is 30*3 matrix
        

        self.idql_net = DDPMIQLLearner.create(
            seed = 42,
            observation_space = self.observation_space,
            action_space = self.action_space,
        )

        self.touch_gpu()
    
    
    def __repr__(self):
        return f"IDQL({self.truck.vid}, {self.driver.pid})"

    def __str__(self):
        return "IDQL"

    def __hash__(self):
        return hash(self.__repr__())

    def touch_gpu(self):
        """touch the gpu to avoid the first time delay"""

        # tf.summary.trace_on(graph=true, profiler=true)
        # ignites manual loading of tensorflow library, \
        # to guarantee the real-time processing of first data in main thread
        init_states = pd.Series(
            np.random.rand(self.truck.observation_numel)
        )  # state must have 30 (speed, throttle, current, voltage) 5 tuple

        # init_states = tf.expand_dims(input_array, 0)  # motion states is 30*2 matrix

        _ = self.actor_predict(init_states)
        self.logger.info(
            f"manual load tf library by calling convert_to_tensor",
            extra=self.dict_logger,
        )

        self.actor_net.ou_noise.reset()

        # warm up the gpu training graph execution pipeline
        if self.buffer.pool.cnt != 0:
            if not self.infer_mode:
                self.logger.info(
                    f"rdpg warm up training!",
                    extra=self.dict_logger,
                )
                (_, _) = self.train()

                self.logger.info(
                    f"rdpg warm up training done!",
                    extra=self.dict_logger,
                )

    def init_checkpoint(self):
        """create or restore from checkpoint"""

        # actor create or restore from checkpoint
        # add checkpoints manager
        self._ckpt_idql_dir = Path(self.data_folder).joinpath(
            "tf_ckpts-"
            + self.__str__()
            + "-"
            + self.truck.vid
            + "-"
            + self.driver.pid
            + "_"
            + "/actor"
        )

        try:
            os.makedirs(self._ckpt_actor_dir)
            self.logger.info(
                "created checkpoint directory for actor: %s",
                self._ckpt_actor_dir,
                extra=self.dict_logger,
            )
        except FileExistsError:
            self.logger.info(
                "actor checkpoint directory already exists: %s",
                self._ckpt_actor_dir,
                extra=self.dict_logger,
            )

    # TODO for infer only mode, implement a method without noisy exploration.
    def actor_predict(
        self, state: pd.Series  # state sequence of the current episode
    ) -> np.ndarray:  # action sequence of the current episode
        """
        sample actions with additive ou noise
        
        input: state is a pd.Series of length 3*10*3/4*50*3 (r*c), output numpy array

        Action outputs and noise object are all row vectors of length 21*17 (r*c), output numpy array
        """

        # We make sure action is within bounds
        # legal_action = np.clip(sampled_actions, action_lower, action_upper)
        # get flat interleaved (not column-wise stacked) tensor from dataframe

        state_flat = None
        try:
            state_flat = jnp.array(state.to_numpy())
        except Exception as e:
            print(f"Exception: {e}")

        states = jnp.expand_dims(
            check_type(state_flat, jnp.ndarray), 0
        )  # motion states is 30*3 matrix

        # sample implicit policy (not the behavior poicly), batch dimension in result is squeezed
        sampled_actions = self.idql_net.sample_implicit_policy(states)
        
        self.logger.info(f"Inference IDQL done!", extra=self.dict_logger)
        # return np.squeeze(sampled_actions)  # ? might be unnecessary
        return sampled_actions


    def sample_minibatch(self):
        """Convert batch type from DataFrames to flattened tensors."""
        if (
            self.buffer.pool.cnt == 0
        ):  # bootstrap for Episode 0 from the current self.observations list
            self.logger.info(
                f"no data in pool, bootstrap from observation_list, "
                f"truck: {self.truck.vid}, driver: {self.driver.pid}.",
                extra=self.dict_logger,
            )
            assert (
                len(self.observations) > 0
            ), "no data in temporary buffer self.observations!"

            # sample from self.observations

            batch_idx = np.random.choice(
                len(self.observations), self.hyper_param.BatchSize  # 4
            )
            observation_samples = [
                self.observations[i] for i in batch_idx
            ]  # a sampled list of Series

            idx = pd.IndexSlice
            state = []
            action = []
            reward = []
            nstate = []
            for (
                observation
            ) in (
                observation_samples
            ):  # each observation is a Series, contiguous storage in rows (already flattened!)
                state.append(
                    observation.loc[
                        idx["state", ["velocity", "thrust", "brake"]]
                    ].values
                )
                action.append(
                    observation.loc[idx["action", self.torque_table_row_names]].values
                )
                reward.append(observation.loc[idx["reward", ["work"]]].values)
                nstate.append(
                    observation.loc[
                        idx["nstate", ["velocity", "thrust", "brake"]]
                    ].values
                )

            # convert to tensors by stacking so that the first dimension is batch_size
            states = tf.convert_to_tensor(np.stack(state), dtype=tf.float32)
            actions = tf.convert_to_tensor(np.stack(action), dtype=tf.float32)
            rewards = tf.convert_to_tensor(np.stack(reward), dtype=tf.float32)
            next_states = tf.convert_to_tensor(np.stack(nstate), dtype=tf.float32)

        else:  # otherwise sample from pool, ignoring the current ongoing episode to reduce complexity
            # TODO combine current episode with pool, need evenly sampling pool and list of observations, then combine
            # get sampling range, if not enough data, batch is small
            self.logger.info(
                f"start sample from pool with size: {self.hyper_param.BatchSize}, "
                f"truck: {self.truck.vid}, driver: {self.driver.pid}.",
                extra=self.dict_logger,
            )

            (
                states,
                actions,
                rewards,
                nstates,
            ) = self.buffer.sample()  # for both mongo and arrow pool

            states = tf.convert_to_tensor(states, dtype=tf.float32)
            actions = tf.convert_to_tensor(actions, dtype=tf.float32)
            rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
            next_states = tf.convert_to_tensor(nstates, dtype=tf.float32)

        return states, actions, rewards, next_states

    def train(self):
        """Train the networks on the batch sampled from the pool."""
        (
            states,
            actions,
            rewards,
            next_states,
        ) = self.sample_minibatch()

        batch = FrozenDict({"actions": actions, 
                            "rewards": rewards, 
                            "observations": states, 
                            "next_observations": next_states})
        
        agent, info = self.agent.update(batch)
        self.agent = agent
        critic_loss = info["critic_loss"]
        actor_loss = info["actor_loss"]
        _ = info["value_loss"]
        return critic_loss, actor_loss


    def soft_update_target(self):
        """update the target networks with Polyak averaging"""
        # it's already done in jaxrl5 agent update
        pass

    # we only calculate the loss

    def save_ckpt(self):
        """TODO Save the checkpoint of the actor, critic and value network in Flax."""
        pass
    
    # We only compute the loss and don't update parameters
    def get_losses(self):
        """Get the losses of the networks on the batch sampled from the pool."""
        (
            states,
            actions,
            rewards,
            next_states,
        ) = self.sample_minibatch()

        batch = FrozenDict({"actions": actions, 
                            "rewards": rewards, 
                            "observations": states, 
                            "next_observations": next_states})
        
        agent, info = self.agent.eval_loss(batch)

        critic_loss = 0.0
        actor_loss = info["actor_loss"]
        return critic_loss, actor_loss



In [ ]:
#| hide
from nbdev.showdoc import show_doc

/Users/x/.pyenv/versions/miniconda3-3.11-24.1.2-0/envs/tspace/lib/python3.11/site-packages/nbdev/doclinks.py:17: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources,importlib
/Users/x/.pyenv/versions/miniconda3-3.11-24.1.2-0/envs/tspace/lib/python3.11/site-packages/pkg_resources/__init__.py:2825: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)


In [ ]:
show_doc(IDQL.__post_init__)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L57){target="_blank" style="float:right; font-size:smaller"}

### IDQL.__post_init__

>      IDQL.__post_init__ ()

*initialize the rdpg agent.

args:
    truck.ObservationNumber (int): dimension of the state space.
    padding_value (float): value to pad the state with, impossible value for observation, action or re*

In [ ]:
show_doc(IDQL.__repr__)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L119){target="_blank" style="float:right; font-size:smaller"}

### IDQL.__repr__

>      IDQL.__repr__ ()

*Return repr(self).*

In [ ]:
show_doc(IDQL.__str__)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L122){target="_blank" style="float:right; font-size:smaller"}

### IDQL.__str__

>      IDQL.__str__ ()

*Return str(self).*

In [ ]:
show_doc(IDQL.__hash__)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L125){target="_blank" style="float:right; font-size:smaller"}

### IDQL.__hash__

>      IDQL.__hash__ ()

*Return hash(self).*

In [ ]:
show_doc(IDQL.touch_gpu)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L128){target="_blank" style="float:right; font-size:smaller"}

### IDQL.touch_gpu

>      IDQL.touch_gpu ()

*touch the gpu to avoid the first time delay*

In [ ]:
show_doc(IDQL.init_checkpoint)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L162){target="_blank" style="float:right; font-size:smaller"}

### IDQL.init_checkpoint

>      IDQL.init_checkpoint ()

*create or restore from checkpoint*

In [ ]:
show_doc(IDQL.actor_predict)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L193){target="_blank" style="float:right; font-size:smaller"}

### IDQL.actor_predict

>      IDQL.actor_predict (state:pandas.core.series.Series)

*sample actions with additive ou noise

input: state is a pd.Series of length 3*10*3/4*50*3 (r*c), output numpy array

Action outputs and noise object are all row vectors of length 21*17 (r*c), output numpy array*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| state | Series | state sequence of the current episode |
| **Returns** | **ndarray** | **action sequence of the current episode** |

In [ ]:
show_doc(IDQL.sample_minibatch)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L225){target="_blank" style="float:right; font-size:smaller"}

### IDQL.sample_minibatch

>      IDQL.sample_minibatch ()

*Convert batch type from DataFrames to flattened tensors.*

In [ ]:
show_doc(IDQL.train)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L302){target="_blank" style="float:right; font-size:smaller"}

### IDQL.train

>      IDQL.train ()

*Train the networks on the batch sampled from the pool.*

In [ ]:
show_doc(IDQL.soft_update_target)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L327){target="_blank" style="float:right; font-size:smaller"}

### IDQL.soft_update_target

>      IDQL.soft_update_target ()

*update the target networks with Polyak averaging*

In [ ]:
show_doc(IDQL.save_ckpt)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L334){target="_blank" style="float:right; font-size:smaller"}

### IDQL.save_ckpt

>      IDQL.save_ckpt ()

*TODO Save the checkpoint of the actor, critic and value network in Flax.*

In [ ]:
show_doc(IDQL.get_losses)

---

[source](https://github.com/Binjian/tspace/blob/main/tspace/agent/idql.py#L339){target="_blank" style="float:right; font-size:smaller"}

### IDQL.get_losses

>      IDQL.get_losses ()

*Get the losses of the networks on the batch sampled from the pool.*

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()